# Estimating At-Work Subtour Frequency

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "atwork_subtour_frequency"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/atwork_subtour_frequency/atwork_subtour_frequency_coefficients.csv
loading spec from output-est-mode/estimation_data_bundle/atwork_subtour_frequency/atwork_subtour_frequency_SPEC.csv
loading from output-est-mode/estimation_data_bundle/atwork_subtour_frequency/atwork_subtour_frequency_values_combined.parquet


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

## Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coefficient_dummy_for_full_time_worker_business1,-7.3750,F
coefficient_dummy_for_full_time_worker_business2,-14.2800,F
coefficient_dummy_for_full_time_worker_eat,-7.2800,F
coefficient_dummy_for_full_time_worker_eat_business,-14.7900,F
coefficient_dummy_for_full_time_worker_maint,-8.0930,F
...,...,...
coefficient_at_work_sub_tour_asc_business2,-2.1337,F
coefficient_at_work_sub_tour_asc_eat,0.8576,F
coefficient_at_work_sub_tour_asc_eat_business,-0.9721,F


## Utility specification

In [5]:
data.spec

,Label,Expression,no_subtours,eat,business1,maint,business2,eat_business
0,util_dummy_for_full_time_worker,pemploy==1,coefficient_dummy_for_full_time_worker_no_subt...,coefficient_dummy_for_full_time_worker_eat,coefficient_dummy_for_full_time_worker_business1,coefficient_dummy_for_full_time_worker_maint,coefficient_dummy_for_full_time_worker_business2,coefficient_dummy_for_full_time_worker_eat_bus...
1,util_dummy_for_non_full_time_worker,pemploy!=1,coefficient_dummy_for_non_full_time_worker_no_...,coefficient_dummy_for_non_full_time_worker_eat,coefficient_dummy_for_non_full_time_worker_bus...,coefficient_dummy_for_non_full_time_worker_maint,coefficient_dummy_for_non_full_time_worker_bus...,coefficient_dummy_for_non_full_time_worker_eat...
2,util_dummy_for_non_workers,"ptype in [4, 5]",coefficient_dummy_for_non_workers_no_subtours,coefficient_dummy_for_non_workers_eat,coefficient_dummy_for_non_workers_business1,coefficient_dummy_for_non_workers_maint,coefficient_dummy_for_non_workers_business2,coefficient_dummy_for_non_workers_eat_business
3,util_medium_hh_income_dummy,income_segment == 2,coefficient_medium_hh_income_dummy_no_subtours,coefficient_medium_hh_income_dummy_eat,coefficient_medium_hh_income_dummy_business1,coefficient_medium_hh_income_dummy_maint,coefficient_medium_hh_income_dummy_business2,coefficient_medium_hh_income_dummy_eat_business
4,util_high_hh_income_dummy,(income_segment > 2) & (income_segment < 5),coefficient_high_hh_income_dummy_no_subtours,coefficient_high_hh_income_dummy_eat,coefficient_high_hh_income_dummy_business1,coefficient_high_hh_income_dummy_maint,coefficient_high_hh_income_dummy_business2,coefficient_high_hh_income_dummy_eat_business
5,util_zero_cars_owned_by_hh_dummy,auto_ownership == 0,coefficient_zero_cars_owned_by_hh_dummy_no_sub...,coefficient_zero_cars_owned_by_hh_dummy_eat,coefficient_zero_cars_owned_by_hh_dummy_business1,coefficient_zero_cars_owned_by_hh_dummy_maint,coefficient_zero_cars_owned_by_hh_dummy_business2,coefficient_zero_cars_owned_by_hh_dummy_eat_bu...
6,util_individual_discretionary_tours_made_by_fu...,@(df.pemploy==1)*df.num_discr_tours,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...
7,util_individual_discretionary_tours_made_by_pa...,@(df.pemploy==2)*df.num_discr_tours,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...,coefficient_individual_discretionary_tours_mad...
8,util_individual_eating_out_tours_made_by_person,num_eatout_tours,coefficient_individual_eating_out_tours_made_b...,coefficient_individual_eating_out_tours_made_b...,coefficient_individual_eating_out_tours_made_b...,coefficient_individual_eating_out_tours_made_b...,coefficient_individual_eating_out_tours_made_b...,coefficient_individual_eating_out_tours_made_b...
9,util_main_shop_escort_tours_allocated_to_full_...,@(df.pemploy==1)*df.num_maint_shop_escort,coefficient_main_shop_escort_tours_allocated_t...,coefficient_main_shop_escort_tours_allocated_t...,coefficient_main_shop_escort_tours_allocated_t...,coefficient_main_shop_escort_tours_allocated_t...,coefficient_main_shop_escort_tours_allocated_t...,coefficient_main_shop_escort_tours_allocated_t...


## Chooser data

In [6]:
data.chooser_data

,model_choice,override_choice,util_dummy_for_full_time_worker,util_dummy_for_non_full_time_worker,util_dummy_for_non_workers,util_medium_hh_income_dummy,util_high_hh_income_dummy,util_zero_cars_owned_by_hh_dummy,util_individual_discretionary_tours_made_by_full_time_worker,util_individual_discretionary_tours_made_by_part_time_worker,...,trPkTotal,trOpRetail,trOpTotal,nmRetail,nmTotal,num_maint_shop_escort,num_joint_discr,num_joint_maint_shop_eat,work_tour_is_SOV,override_choice_code
tour_id,,,,,,,,,,,,,,,,,,,,,
2966594,no_subtours,eat,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.119016,6.446184,9.035333,5.256966,6.831275,0,0,0,False,1
3046667,eat,eat,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.771767,5.298242,7.602426,6.443514,7.965548,0,0,0,False,1
3048143,no_subtours,eat,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.771767,5.298242,7.602426,6.443514,7.965548,0,0,0,True,1
3177498,no_subtours,eat,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,True,1
3191848,eat,maint,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.278544,1.239060,3.280429,4.284159,6.249753,0,0,0,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308156039,no_subtours,eat,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,3.086628,5.647842,0,0,0,False,1
308227666,no_subtours,maint,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.890189,4.493279,7.643024,5.529241,8.876000,0,0,0,True,3
308260138,no_subtours,eat,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.614481,3.838266,7.011926,5.169891,8.595128,0,0,0,False,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [7]:
model.estimate(method='SLSQP', maxiter=900)

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coefficient_at_work_sub_tour_asc_business1,-0.350395,-0.350395,-0.5372,-50.0,50.0,0.0,0
coefficient_at_work_sub_tour_asc_business2,-2.211281,-2.211281,-2.1337,-50.0,50.0,0.0,0
coefficient_at_work_sub_tour_asc_eat,0.440311,0.440311,0.8576,-50.0,50.0,0.0,0
coefficient_at_work_sub_tour_asc_eat_business,-0.860366,-0.860366,-0.9721,-50.0,50.0,0.0,0
coefficient_at_work_sub_tour_asc_maint,-0.423469,-0.423469,-0.6198,-50.0,50.0,0.0,0
...,...,...,...,...,...,...,...
coefficient_zero_cars_owned_by_hh_dummy_business1,-0.633755,-0.633755,-0.3391,-50.0,50.0,0.0,0
coefficient_zero_cars_owned_by_hh_dummy_business2,0.000000,0.000000,0.0000,0.0,0.0,0.0,1
coefficient_zero_cars_owned_by_hh_dummy_eat,0.000000,0.000000,0.0000,0.0,0.0,0.0,1


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


┣                x: coefficient_at_work_sub_tour_asc_business1             -0.350395
┃                   coefficient_at_work_sub_tour_asc_business2             -2.211281
┃                   coefficient_at_work_sub_tour_asc_eat                    0.440311
┃                   coefficient_at_work_sub_tour_asc_eat_business          -0.860366
┃                   coefficient_at_work_sub_tour_asc_maint                 -0.423469
┃                                                                             ...   
┃                   coefficient_zero_cars_owned_by_hh_dummy_business1      -0.633755
┃                   coefficient_zero_cars_owned_by_hh_dummy_business2       0.000000
┃                   coefficient_zero_cars_owned_by_hh_dummy_eat             0.000000
┃                   coefficient_zero_cars_owned_by_hh_dummy_eat_business   -0.847707
┃                   coefficient_zero_cars_owned_by_hh_dummy_maint          -0.020836
┃                   Length: 110, dtype: float64
┣          logloss: 0.8125481383373976
┣        d_logloss: coefficient_at_work_sub_tour_asc_business1             -0.000022
┃                   coefficient_at_work_sub_tour_asc_business2             -0.000008
┃                   coefficient_at_work_sub_tour_asc_eat                    0.000069
┃                   coefficient_at_work_sub_tour_asc_eat_business          -0.000006
┃                   coefficient_at_work_sub_tour_asc_maint                 -0.000033
┃                                                                             ...   
┃                   coefficient_zero_cars_owned_by_hh_dummy_business1      -0.000013
┃                   coefficient_zero_cars_owned_by_hh_dummy_business2       0.000000
┃                   coefficient_zero_cars_owned_by_hh_dummy_eat             0.000000
┃                   coefficient_zero_cars_owned_by_hh_dummy_eat_business   -0.000081
┃                   coefficient_zero_cars_owned_by_hh_dummy_maint          -0.000024
┃                   Length: 110, dtype: float64
┣              nit: 100
┣             nfev: 104
┣             njev: 100
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(microseconds=874759)
┣           method: 'SLSQP'
┣          n_cases: 5947
┣ iteration_number: 100
┣          loglike: -4832.223778692503

### Estimated coefficients

In [8]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coefficient_at_work_sub_tour_asc_business1,-0.350,1.10e+03,-0.00,,0.00,
coefficient_at_work_sub_tour_asc_business2,-2.21,2.21e+03,-0.00,,0.00,
coefficient_at_work_sub_tour_asc_eat,0.440,NA,NA,,0.00,
coefficient_at_work_sub_tour_asc_eat_business,-0.860,495.,-0.00,,0.00,
coefficient_at_work_sub_tour_asc_maint,-0.423,NA,NA,,0.00,
coefficient_auto_accessibility_to_retail_for_work_taz_business1,0.0782,NA,NA,,0.00,
coefficient_auto_accessibility_to_retail_for_work_taz_business2,0.296,NA,NA,,0.00,
coefficient_auto_accessibility_to_retail_for_work_taz_eat,0.137,NA,NA,,0.00,
coefficient_auto_accessibility_to_retail_for_work_taz_eat_business,-0.263,NA,NA,,0.00,


# Output Estimation Results

In [9]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [10]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [11]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coefficient_dummy_for_full_time_worker_business1,-7.402415,F
1,coefficient_dummy_for_full_time_worker_business2,-13.824988,F
2,coefficient_dummy_for_full_time_worker_eat,-7.551797,F
3,coefficient_dummy_for_full_time_worker_eat_bus...,-14.974823,F
4,coefficient_dummy_for_full_time_worker_maint,-8.063977,F
...,...,...,...
127,coefficient_at_work_sub_tour_asc_business2,-2.211281,F
128,coefficient_at_work_sub_tour_asc_eat,0.440311,F
129,coefficient_at_work_sub_tour_asc_eat_business,-0.860366,F
130,coefficient_at_work_sub_tour_asc_maint,-0.423469,F
